# Module 2, Activity 1: Exploring Prompt Variations

When it comes to creating AI applications, the secret sauce is in the prompt.  Prompt engineering is as much of an art as it is a science.  But you will see in this notebook that there are some rules you can follow to write effective prompts.  We begin by loading up our typical code.

In [ ]:
import json
from pprint import pprint
import boto3

from langchain.chains import LLMChain
from langchain_aws import ChatBedrock
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda

In [ ]:
session = boto3.session.Session()
region = session.region_name

In [ ]:
def get_data_from_s3(bucket_name, key):
    s3 = boto3.client(
        's3',
        region_name=region,
    )
    response = s3.get_object(Bucket=bucket_name, Key=key)
    data = response['Body'].read().decode('utf-8')

    return data

## Analyzing text

We are going to focus on analyzing raw text, however, what we learn below will be applicable to any of the three tracks for this course.

In [ ]:
s3_data = get_data_from_s3("...", "constitution.txt")
s3_data[0:200]

## Create our basic chain

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant."
)
human_prompt = HumanMessagePromptTemplate.from_template(
    "{input}"
)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=region,
    temperature=0.5,
    max_tokens=1000,
)

chain = prompt | llm | StrOutputParser()

## Zero-Shot Prompting

The most basic type of prompting is called "zero-shot prompting," which is when you provide the model with just the question or instruction and expect it to get the answer just based on its pre-trained knowledge.  No examples or additional context are provided.  Let's try it:

In [ ]:
zero_shot_prompt = f"Provide an analysis of {s3_data}"

chain.invoke({"input": zero_shot_prompt})

## Few-Shot Prompting

While that did alright, we can do better if we can provide a few examples within the prompt to guide the model's behavior.  This is called "few-shot prompting."  By demonstrating the format, style, or type of answer you expect, the model can better understand and mimic that structure in its response.

In [ ]:
few_shot_prompt = f"""
    You are provided with {s3_data} and given the following:
    Example 1:
    Q: What is the significance of the Constitution of the United States?
    A: The Constitution is the supreme law of the United States, establishing the framework for the federal government and protecting individual rights.
    
    Example 2:
    Q: How does the Constitution implement checks and balances?
    A: It divides power among three branches—executive, legislative, and judicial—ensuring that no single branch dominates the government.
    
    Now, answer the following:
    Q: Provide an analysis of the Constitution of the United States.
    A:
"""

chain.invoke({'input': few_shot_prompt})

## Chain-of-Thought (COT) Prompts

Chain-of-thought prompting is a technique where you guide the model to break down its reasoning process into sequential steps before arriving at the final answer. Instead of generating a direct answer in one go, you instruct the model to "think aloud" by detailing intermediate steps.  This can lead to more thorough and accurate responses, especially for complex or multi-step problems.  It works particularly well with more sophisticated models.

In [ ]:
chain_of_thought_prompt = f"""
You are an expert in constitutional law.  You have been provided {s3_data}.
Please analyze this data by following these steps:
Step 1: Summarize the primary purpose and structure of the Constitution.
Step 2: Explain the system of checks and balances and why it is crucial.
Step 3: Discuss how the Constitution has influenced modern governance and legal interpretations.
Provide your detailed analysis:
"""

chain.invoke({'input': chain_of_thought_prompt})

## Best Practices for Prompt Engineering

Notice that we used statements like "you are an expert in..."  Informing the LLM how they should respond within the prompt is considered to be good prompt engineering practice.  But there are many other things you should consider adding to your prompts.  Here are some general guidelines taken from [this website](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-the-openai-api):

- Use the latest models (noting that many popular models are updated regularly...be sure you have the most recent version)
- Put the instructions at the beginning of the prompt and use delimiters like `####` or `""""` to separate the instruction and context.
- Be specific, descriptive, and as detailed as possible about the desired context, outcome, length, format, style, etc.
- Articulate the desired output format through examples
- When possible, do not use imprecise descriptions
- Don't just say what NOT to do...say what to do instead

Let's now give some of these a try with a different problem.  We will start by refreshing our chain...

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant."
)
human_prompt = HumanMessagePromptTemplate.from_template(
    "{input}"
)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [ ]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=region,
    temperature=0.5,
    max_tokens=1000,
)

chain = prompt | llm | StrOutputParser()

## Experimenting with Prompts

Below are prompts all asking about the same thing but worded differently.  Please experiment with them to see the types of answers you get and try to make them better.

In [ ]:
zero_shot_prompt = "Explain the benefits of using generative AI in software development."

few_shot_prompt = """
    You will be provided some example questions after the #### delimiter.  You will then be asked a question after the next
    #### delimeter.  Answer the question in two sentences.

    ####
    Example 1:
    Q: What is generative AI?
    A: Generative AI is a type of artificial intelligence that can create new content by learning from a dataset.
    
    Example 2:
    Q: How can generative AI assist software engineers?
    A: It can automate repetitive coding tasks and generate documentation.

    ####
    Now answer:
    Q: Explain the benefits of using generative AI in software development.
    A:
"""

chain_of_thought_prompt = """
    You are a senior software engineer.  You will be provided a question and some steps to follow after the #### delimeter.
    Provide your answer with minimal jargon like you are explaining it to a 14 year old.  Your total answer should not go
    over 3 benefits with a sentence explaining each.
    ####
    Please explain the benefits of using generative AI in software development by breaking your response into steps:
    Step 1: List the main benefits.
    Step 2: Explain why each benefit is important.
"""

In [ ]:
chain.invoke({"input": zero_shot_prompt})

In [ ]:
chain.invoke({"input": few_shot_prompt})

In [ ]:
chain.invoke({"input": chain_of_thought_prompt})